# **Delivery Optimization**
---
**Author:** Hai Pham

## 1. Business Problem

Given the boom in e-commerce, delivery service plays a vital part in creating competitive advantages and is key to minizing cost for businesses as well. E-commerce firms, whose thousands of warehouses are distributed nationwide, always want to minimize the distance to deliver goods from a warehouse to a customer. That being said, selecting which warehouse to take goods and deliver to a specific customer is the first thing to consider.

With that idea in mind, I leverage Python to carry out delivery optimization from warehouses and customers. Due to the availability of data, I decided to choose Costco as a proxy for business with hundreds of warehouses and Starbuck stores as a sample of customers. These dataset were obtained from Kaggle.com.

## 2. Data Loading

Let's get started with package import

In [1]:
import pandas as pd
from geopy import distance
import numpy as np

The data sets are obtained from Kaggle and then uploaded to dropbox so we can load it from url.

In [2]:
costco_data_url = 'https://www.dropbox.com/s/6ovgn14250jrlbm/costco_data.csv?dl=1'
costco_dat = pd.read_csv(costco_data_url, na_values=np.NaN)

starbuck_data_url = 'https://www.dropbox.com/s/yxsd7ii34iq863o/starbuck_data.csv?dl=1'
starbuck_dat = pd.read_csv(starbuck_data_url, na_values=np.NaN)

Get a general information of Costco dataset

In [3]:
costco_dat.info(),

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                782 non-null    int64  
 1   stlocID                   782 non-null    int64  
 2   phone                     734 non-null    object 
 3   fax                       0 non-null      float64
 4   address1                  782 non-null    object 
 5   city                      782 non-null    object 
 6   state                     750 non-null    object 
 7   country                   782 non-null    object 
 8   zipCode                   777 non-null    object 
 9   manager                   753 non-null    object 
 10  latitude                  782 non-null    float64
 11  longitude                 782 non-null    float64
 12  parentGeoNodeID           782 non-null    int64  
 13  active                    782 non-null    int64  
 14  languageID

(None,)

We can see the Costco dataset includes 782 rows and 37 columns. Each row represents a Costco store and each column represents a feature of store.

Take a look at the first 5 rows of this dataset.

In [4]:
costco_dat.head()

,Unnamed: 0,stlocID,phone,fax,address1,city,state,country,zipCode,manager,...,tireCenterHours,gasPrices,coreServices,specialtyDepartments,locationName,isShipToWarehouse,isWarehousePickup,enableShipToHome,isBusinessWarehouse,closeDate
0,0,1,(206) 622-3136,NaN,4401 4TH AVE S,SEATTLE,WA,US,98134-2389,JOHN BARTLETT,...,"[{'title': 'Mon-Fri.', 'code': 'open', 'time':...","{'warehouseid': '1', 'regular': '2.799', 'prem...","[{'name': 'Gas Station', 'localizedName': 'Gas...","[{'name': 'Auto Buying Program', 'localizedNam...",Seattle,True,True,False,False,NaN
1,1,2,(503) 252-2243,NaN,4849 NE 138TH AVE,PORTLAND,OR,US,97230-3401,GENE MORMON,...,"[{'title': 'Mon-Fri.', 'code': 'open', 'time':...","{'warehouseid': '2', 'diesel': '2.899', 'regul...","[{'name': 'Gas Station', 'localizedName': 'Gas...","[{'name': 'ATM', 'localizedName': 'ATM', 'phon...",Portland,True,True,False,False,NaN
2,2,6,(206) 575-9191,NaN,400 COSTCO DR STE 150,TUKWILA,WA,US,98188-4808,TODD YOUNG,...,"[{'title': 'Mon-Fri.', 'code': 'open', 'time':...","{'warehouseid': '6', 'diesel': '2.959', 'regul...","[{'name': 'Gas Station', 'localizedName': 'Gas...","[{'name': 'Auto Buying Program', 'localizedNam...",Tukwila,True,True,False,False,NaN
3,3,8,(425) 827-1693,NaN,8629 120TH AVE NE,KIRKLAND,WA,US,98033-5865,CHRIS DELONG,...,"[{'title': 'Mon-Fri.', 'code': 'open', 'time':...","{'warehouseid': '8', 'regular': '2.739', 'prem...","[{'name': 'Gas Station', 'localizedName': 'Gas...","[{'name': 'Auto Buying Program', 'localizedNam...",Kirkland,True,True,False,False,NaN
4,4,9,(503) 644-7615,NaN,15901 SW JENKINS RD,ALOHA,OR,US,97006-5098,ALEX RYAN,...,"[{'title': 'Mon-Fri.', 'code': 'open', 'time':...","{'warehouseid': '9', 'regular': '2.759', 'prem...","[{'name': 'Gas Station', 'localizedName': 'Gas...","[{'name': 'Auto Buying Program', 'localizedNam...",Aloha,True,True,False,False,NaN


Get a general information of Starbuck dataset:

In [5]:
starbuck_dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25600 entries, 0 to 25599
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           25600 non-null  object 
 1   Store Number    25600 non-null  object 
 2   Store Name      25600 non-null  object 
 3   Ownership Type  25600 non-null  object 
 4   Street Address  25598 non-null  object 
 5   City            25585 non-null  object 
 6   State/Province  25600 non-null  object 
 7   Country         25600 non-null  object 
 8   Postcode        24078 non-null  object 
 9   Phone Number    18739 non-null  object 
 10  Timezone        25600 non-null  object 
 11  Longitude       25599 non-null  float64
 12  Latitude        25599 non-null  float64
dtypes: float64(2), object(11)
memory usage: 2.5+ MB


The Starbuck dataset contains 25,600 rows, each of which represents a Starbuck store, and 13 columns, each of which is a feature of store. Also, let's skim a first few rows of this dataset.

In [6]:
starbuck_dat.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


## 3. Data Cleaning

Given the nature of simulation, I decided to cover only Costco warehouses and Starbuck stores in the US. Let's filter US-based address from our data sets. Also, due to limited CPU processing power, a sample of 100 Starbuck stores are taken for this analysis.

In [7]:
costco_US = costco_dat[costco_dat['country'] =='US']
costco_US.shape

(543, 37)

In [8]:
starbuck_US = starbuck_dat[starbuck_dat['Country'] == 'US']
starbuck_US.shape

(13608, 13)

In [9]:
starbuck_US_100 = starbuck_US.sample(n = 100, random_state = 5)

Now let's filter variables necessary for modeling step:

In [10]:
costco_US_clean = costco_US.loc[:, ['stlocID', 'latitude', 'longitude']]
starbuck_US_clean = starbuck_US_100.loc[:, ['Store Number', 'Latitude', 'Longitude']]

## 4. Modeling

The algorithm for this optimization problem can be explained as follows: 

- Use `for` loop to pair each Starbuck store with all Costco warehouses and calculate their associated distance based on their latitude and longitude.
- The model paired 100 Starbuck stores with all 543 Costco warehouses, leading to 54,300 pairs. I then sorted this output into ascending order of distance to come up with Costco warehouse that is closest to each Starbuck store, obtaining a final outcome of 100 pairs. 
- Costco warehouse in each pair was the one selected to take goods and deliver to its associated Starbuck store.

In [ ]:
# Construct a dictionary to store results
output = {'Starbuck_store_number': [],
         'Costco_warehouse_id': [],
         'Distance': []}


# Use for loop to calculate the distance from each Starbuck store to all Costco warehouses.
for starbuck_id in starbuck_US_clean['Store Number']:
    for costco_id in costco_US_clean['stlocID']:
        starbuck_store = starbuck_US_clean[starbuck_US_clean['Store Number'] == starbuck_id]
        costco_warehouse = costco_US_clean[costco_US_clean['stlocID'] == costco_id]

        starbuck_store_location = (starbuck_store['Latitude'].values, starbuck_store['Longitude'].values)
        costco_warehouse_location = (costco_warehouse['latitude'].values, costco_warehouse['longitude'].values)

        dist = distance.distance(starbuck_store_location, costco_warehouse_location).miles

        output['Starbuck_store_number'].append(starbuck_id)
        output['Costco_warehouse_id'].append(costco_id)
        output['Distance'].append(dist)
        
# Sort the output into ascending order of distance from each Starbuck store to all Costco warehouses, then pick the first row in each subgroup.
output_dataframe = pd.DataFrame(output)
output_dataframe = output_dataframe.sort_values(by=['Starbuck_store_number','Distance'], ascending = True)
final_output = output_dataframe.groupby('Starbuck_store_number').head(1)

That took care of modeling. Let's take a look at the first 10 rows and the shape of our final output:

In [ ]:
display(final_output.head(10),
        final_output.shape)

## 5. Conclusion

This is a simulation to build the algorithm for optimization problem. If greater CPU processing power is available, we can input all data of Costco and Starbuck stores into the model to have a holistic approach.    

Based on the insights from this analysis, Costco can optimize the delivery to Starbuck stores in terms of distances going forward.